# Zero to Hero

## 1.08 Unos buenos hiperparametros

Se aplicará el modelo generado por rpart a los datos nuevos y se genera la salida para Kaggle

In [1]:
rm( list=ls())
gc()

library( "data.table")   #cargo la libreria  data.table
library( "rpart")  #cargo la libreria  rpart
library( "rpart.plot")
require("caret")

options(repr.plot.width=20, repr.plot.height=10) 
setwd("~/lab1/") #Aqui se debe poner la ruta de la PC local

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604257,32.3,1291145,69,1291145,69.0
Vcells,1086175,8.3,8388608,64,1633205,12.5


Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice



In [2]:
set.seed(999953)
dataset <- fread("./datasets/competencia1_2022_v2.csv")   #cargo el dataset #cargo el dataset
#creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
dataset[ , clase_binaria :=  ifelse( clase_ternaria=="CONTINUA", "NO", "SI" ) ]
datasettrain <- dataset[ foto_mes==202101 ]
train_rows <- createDataPartition(datasettrain$clase_binaria, p= 0.66,list= FALSE)
dtrain <- datasettrain[c(train_rows)]
dapply <- datasettrain[-c(train_rows)]
nrow( dtrain )
ncol( dtrain )

[1] 106487

[1] 161

defino unos parametros interesantes
<br>
los hiperparametros van en una lista
<br>
notar la forma en que esos parametros se pasan a la funcion rpart

In [3]:
nrow( datasettrain )
nrow( dtrain )
nrow( dapply )

[1] 161342

[1] 106487

[1] 54855

In [4]:
param1  <- list("cp"= -0.937,
               "minsplit"=  1143,
               "minbucket"= 1,
               "maxdepth"= 9
              )

param2  <- list("cp"= 0.000001,
               "minsplit"=  18,
               "minbucket"= 3,
               "maxdepth"= 10
              )

In [5]:
#genero el modelo
modelo <-  rpart::rpart(formula= "clase_binaria ~ . -clase_ternaria" ,
                        data= dtrain,
                        xval= 0,
                        control= param1)
#print(modelo)

In [14]:

#vim <- varImp(modelo,useModel=T)
vim <- varImp(modelo, scale = TRUE, useModel = TRUE, nonpara = TRUE)
vim

caret::varImp(modelo) %>% 
  rownames_to_column() %>% 
  arrange(desc(Overall)) %>% 
  slice(1:10)



#summary(modelo)

,Overall
,<dbl>
active_quarter,39.06643840
active_vs_mov,8.46916528
avg_visa_master,1.53650794
catm_trx_other,0.98432539
ccaja_ahorro,0.02921448
ccajas_consultas,3.16112269
ccajas_depositos,0.96151259
ccajas_extracciones,1.25718584
ccajas_otras,1.18576436


ERROR: Error in caret::varImp(rf_mod) %>% rownames_to_column() %>% arrange(desc(Overall)) %>% : could not find function "%>%"


Aplico el modelo a los datos dapply  pidiendo que me devuelva probabildades

In [7]:
prediccion  <- predict( modelo, dapply, type = "prob")

Genero la salida

In [8]:
prob_baja2  <- prediccion[, "SI"]
Predicted   <- ifelse( prob_baja2 > 0.032, 1, 0 )  # 0.025 es la probabilidad de corte

entrega  <-  as.data.table( list( "numero_de_cliente"=dapply$numero_de_cliente, "Predicted"=Predicted)  )

#creo la carpeta donde va el experimento
dir.create( "./exp/",  showWarnings = FALSE ) 
dir.create( "./exp/ZH2018/", showWarnings = FALSE )

#genero el archivo para Kaggle
fwrite( entrega, 
        file= "./exp/ZH2018/para_Kaggle_0108_3.csv",
        sep=  "," )

¿Cuánto le da la ganancia en el Public Leaderboard?

In [9]:
preds <- predict(modelo, dapply, type = "class")
#preds

#confusionMatrix(preds,dtrain$clase_ternaria)
#table(preds)

#  BAJA+1   BAJA+2 CONTINUA 
#       4       14   161324 

confusionMatrix(as.factor(dapply$clase_binaria), as.factor(preds), positive = "1")


ERROR: Error in `[.default`(data, , positive): subscript out of bounds


In [ ]:
#nrow(dtrain)
#preds

In [ ]:
#library(ROCR)
#preds  <- prediction(dapply$clase_ternaria), as.factor(preds))
#roc = performance(preds, measure="tpr", x.measure="fpr")
#plot(roc, col="orange", lwd=2) 
#lines(x=c(0, 1), y=c(0, 1), col="red", lwd=2)